Introducing kids in the structural lifecycle framework
==========================================

Blundell et. al. (2016)
-------------------------

State space includes:
- age of younghest child
- presense of children, i.e., mother yes/no

Utility includes:
- presense of children interacted with education level
- age of youngest child binned into 0-2, 3-5, 6-10, 11+ (p. 1726)

Other:
- exogenous process of child arriving depends on age and education
- probability of child arriving is set to zero after 43 years of age
- p. 1729 says that child lives with mother until 18 years of age => children above 18 years of age are irrelevant?

Adda, Dustman, Stevens (2017)
-------------------------------------

State space:
- number of children
- age of younghest child

Utility:
- a lot of interactions: husband, employment status, occupation type, etc.
- it matters if one or two children
- age bins: 0-3, 4-6, 7-9

Other:
- having a child is a choice
- variable updated at the beginning of the period, i.e., assumption children arrive at the beginning of the period and have age zero
- unobserved heterogeneity in the taste for children: correlated with ability and taste for leisure
- unobserved heterogeneity in potential infertility, orthogonal to other heterogeneity components

Implementation
-------------------

Goal:
 - most needed! - information on age of younghest child
 
Achive while minimising computational cost!

Challenges:
 - no way to avoide recording age of younghest child
 - no obvious way of decreasing state space size by using bins for ages: age bins do not contain all information necessary to determine child nodes in the backward induction procedure

Potential solution:
 - implement as many constraints as possible in order to reduce number of states, e.g., women over 40 years of age do not get kids any more, we assume women do not get more than 3 kids, etc.
 - optimize code base as much as possible

In [1]:
from soepy.pre_processing.model_processing import read_model_params_init
from soepy.pre_processing.model_processing import read_model_spec_init

In [2]:
model_params_df, model_params = read_model_params_init("toy_model_init_file_03_3types.pkl")
model_spec = read_model_spec_init("model_spec_init.yml", model_params_df)

In [3]:
model_spec

model_specification(num_periods=40, delta=0.98, mu=-0.56, benefits=4.0, educ_max=12, educ_min=10, low_bound=10, middle_bound=11, high_bound=12, seed_sim=102, num_agents_sim=9000, seed_emax=635, num_draws_emax=500, educ_range=3, num_types=3)

In [4]:
is_expected = False

State Space
---------------

In [5]:
import numba

In [6]:
import numpy as np
import pandas as pd

In [7]:
from soepy.shared.shared_constants import MISSING_INT, NUM_CHOICES, INVALID_FLOAT, HOURS
from soepy.solve.solve_auxiliary import pyth_create_state_space

In [8]:
kids_ages = np.arange(-1,12)

Coding of `age_kid` state space variable:
- `-1` - no kid
- `0` - kid was just born. Meaning: in period `t-1` the random process resulted in the woman receiving a child, so she gets a baby in the current period.
- `1+` - age of younghest child

Conventions:
- in most periods a new child can arrive.
- if a new child arrives when there were no children before, in `t-1`, `age_kid` is `-1` and `arrival` = 1, such that in the current period, `age_kid` becomes `0` and `arrival` can again take the values 0 or 1 with respective probabilities.
- if a new child arrives when there already was a child, in `t-1`, `age_kid` is some number between 0 and 10 and `arrival` = 1, such that in the current period, `age_kid` becomes `0` and `arrival` can again take the values 0 or 1 with respective probabilities.

In [9]:
@numba.jit(nopython=True)
def pyth_create_state_space(model_spec):
    """Create state space object.

    The state space consists of all admissible combinations of the following components:
    period, years of education, lagged choice, full-time experience (F),
    and part-time experience (P).

    :data:`states` stores the information on states in a tabular format.
    Each row of the table corresponds to one admissible state space point
    and contains the values of the state space components listed above.
    :data:`indexer` is a multidimensional array where each component
    of the state space corresponds to one dimension. The values of the array cells
    index the corresponding state space point in :data:`states`.
    Traversing the state space requires incrementing the indices of :data:`indexer`
    and selecting the corresponding state space point component values in :data:`states`.

    Parameters
    ----------
    model_spec: namedtuple
        Namedtuple containing all fixed parameters describing the model and its
         state space that are relevant for running a simulation.

    Returns
    -------
    states : np.ndarray
        Array with shape (num_states, 5) containing period, years of schooling,
        the lagged choice, the years of experience in part-time, and the
        years of experience in full-time employment.
    indexer : np.ndarray
        A matrix where each dimension represents a characteristic of the state space.
        Switching from one state is possible via incrementing appropriate indices by 1.

    Examples
    --------
    >>> from collections import namedtuple
    >>> model_spec = namedtuple(
    ...     "model_specification", "num_periods educ_range educ_min num_types"
    ... )
    >>> model_spec = model_spec(10, 3, 10, 2)
    >>> NUM_CHOICES = 3
    >>> states, indexer = pyth_create_state_space(
    ...     model_spec
    ... )
    >>> states.shape
    (2220, 6)
    >>> indexer.shape
    (10, 3, 3, 10, 10, 2)
    """
    data = []

    # Array for mapping the state space points (states) to indices
    shape = (
        model_spec.num_periods,
        model_spec.educ_range,
        NUM_CHOICES,
        model_spec.num_periods,
        model_spec.num_periods,
        model_spec.num_types,
        kids_ages.shape[0],
    )

    indexer = np.full(shape, MISSING_INT)

    # Initialize counter for admissible state space points
    i = 0

    # Loop over all periods / all ages
    for period in range(model_spec.num_periods):
        
        # Loop over all types
        for type_ in range(model_spec.num_types):
            
            # Loop over all kids ages that are recorded
            for age_kid in kids_ages:
                # Assumption: 1st kid is born no earlier than in period zero,
                # i.e., in the current setup, no earlier than age 17.
                # Can be relaxed, e.g., we assume that 1st kid can arrive earliest when
                # a woman is 16 years old, the condition becomes:
                # if age_kid > period + 1.
                if age_kid > period:
                    continue
                # Make sure that women above 42 do not get kids
                # For periods corresponding to ages > 42, the `age_kid`
                # state space component can only take values -1, for no child ever,
                # 11, for a child above 11, and 0 - 10 in such a fashion that no
                # birth after 42 years of age is possible.
                if (period > 24 and 0 <= age_kid <= min(period - 25, 10)):
                    continue

                # Loop over all possible initial conditions for education
                for educ_years in range(model_spec.educ_range):

                    # Check if individual has already completed education
                    # and will make a labor supply choice in the period
                    if educ_years > period:
                        continue

                    # Loop over all admissible years of experience accumulated in full-time
                    for exp_f in range(model_spec.num_periods):

                        # Loop over all admissible years of experience accumulated
                        # in part-time
                        for exp_p in range(model_spec.num_periods):

                            # The accumulation of experience cannot exceed time elapsed
                            # since individual entered the model
                            if exp_f + exp_p > period - educ_years:
                                continue

                            # Add an additional entry state
                            # [educ_years + model_params.educ_min, 0, 0, 0]
                            # for individuals who have just completed education
                            # and still have no experience in any occupation.
                            if period == educ_years:

                                # Assign an additional integer count i
                                # for entry state
                                indexer[period, educ_years, 0, 0, 0, type_, age_kid] = i

                                # Record the values of the state space components
                                # for the currently reached entry state
                                row = [
                                    period,
                                    educ_years + model_spec.educ_min,
                                    0,
                                    0,
                                    0,
                                    type_,
                                    age_kid,
                                ]

                                # Update count once more
                                i += 1

                                data.append(row)

                            else:

                                # Loop over the three labor market choices, N, P, F
                                for choice_lagged in range(NUM_CHOICES):

                                    # If individual has only worked full-time in the past,
                                    # she can only have full-time (2) as lagged choice
                                    if (choice_lagged != 2) and (
                                        exp_f == period - educ_years
                                    ):
                                        continue

                                    # If individual has only worked part-time in the past,
                                    # she can only have part-time (1) as lagged choice
                                    if (choice_lagged != 1) and (
                                        exp_p == period - educ_years
                                    ):
                                        continue

                                    # If an individual has never worked full-time,
                                    # she cannot have that lagged activity
                                    if (choice_lagged == 2) and (exp_f == 0):
                                        continue

                                    # If an individual has never worked part-time,
                                    # she cannot have that lagged activity
                                    if (choice_lagged == 1) and (exp_p == 0):
                                        continue

                                    # If an individual has always been employed,
                                    # she cannot have non-employment (0) as lagged choice
                                    if (choice_lagged == 0) and (
                                        exp_f + exp_p == period - educ_years
                                    ):
                                        continue

                                    # Check for duplicate states
                                    if (
                                        indexer[
                                            period,
                                            educ_years,
                                            choice_lagged,
                                            exp_p,
                                            exp_f,
                                            type_,
                                            age_kid,
                                        ]
                                        != MISSING_INT
                                    ):
                                        continue

                                    # Assign the integer count i as an indicator for the
                                    # currently reached admissible state space point
                                    indexer[
                                        period,
                                        educ_years,
                                        choice_lagged,
                                        exp_p,
                                        exp_f,
                                        type_,
                                        age_kid,
                                    ] = i

                                    # Update count
                                    i += 1

                                    # Record the values of the state space components
                                    # for the currently reached admissible state space point
                                    row = [
                                        period,
                                        educ_years + model_spec.educ_min,
                                        choice_lagged,
                                        exp_p,
                                        exp_f,
                                        type_,
                                        age_kid,
                                    ]

                                    data.append(row)

        states = np.array(data)

    # Return function output
    return states, indexer

In [10]:
states, indexer = pyth_create_state_space(model_spec)

In [11]:
states.shape

(1772145, 7)

In [41]:
%timeit pyth_create_state_space(model_spec)

1.75 s ± 22.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Construct covariates
-------------------------

In [13]:
def construct_covariates(states, model_spec):
    """Construct a matrix of all the covariates
    that depend only on the state space.

    Parameters
    ---------
    states : np.ndarray
        Array with shape (num_states, 5) containing period, years of education,
        the lagged choice, years of experience in part-time and in full-time
         employment of the agent.

    Returns
    -------
    covariates : np.ndarray
        Array with shape (num_states, number of covariates) containing all additional
        covariates, which depend only on the state space information.

    """
    
    # Level of education derived from years of education
    educ_years = pd.Series(states[:, 1])
    educ_level = pd.cut(
        educ_years,
        bins=[0, model_spec.low_bound, model_spec.middle_bound, model_spec.high_bound],
        labels=[0, 1, 2],
    ).to_numpy()
    
    # Is a mother dummy derived from kids age
    # age_kid = pd.Series(states[:, 6])
    # is_mother = age_kid.where(age_kid == -1, 1)
    # is_mother = is_mother.where(is_mother == 1, 0)
    
    # Bins of age of younghest child based on kids age
    # bin 0 corresponds to no kid, remaining bins as in Blundell
    # 0-2, 3-5, 6-10, 11+
    age_kid = pd.Series(states[:, 6])
    bins = pd.cut(
        age_kid,
        bins = [-2, -1, 2, 5, 10, 11],
        labels = [0, 1, 2, 3, 4],
    ).to_numpy()
    
    covariates = np.column_stack(
        (
            educ_level,
            # is_mother,
            bins
        )
    )

    return covariates

In [14]:
covariates = construct_covariates(states, model_spec)

In [15]:
covariates.shape

(1772145, 2)

Calculate utility
------------------

In [16]:
def calculate_utility_components(
    model_params, model_spec, states, covariates, is_expected
):
    """Calculate utility components for all choices given state, period, and shocks.

    Parameters
    ----------
    model_params : namedtuple
        Contains all parameters of the model including information on dimensions
        (number of periods, agents, random draws, etc.) and coefficients to be
        estimated.
    states : np.ndarray
        Array with shape (num_states, 5) containing period, years of schooling,
        the lagged choice, the years of experience in part-time, and the
        years of experience in full-time employment.
    covariates: np.ndarray
        Array with shape (num_states, number of covariates) containing all additional
        covariates, which depend only on the state space information.
    is_expected: bool
        A boolean indicator that differentiates between the human capital accumulation
        process that agents expect (is_expected = True) and that the market generates
        (is_expected = False)

    Returns
    -------
    log_wage_systematic : array
        One dimensional array with length num_states containing the part of the wages
        at the respective state space point that do not depend on the agent's choice,
        nor on the random shock.
    non_consumption_utilities : np.ndarray
        Array of dimension (num_states, num_choices) containing the utility
        contribution of non-pecuniary factors.

    """
    log_wage_systematic = calculate_log_wage_systematic(
        model_params, states, covariates, is_expected
    )

    non_consumption_utility = calculate_non_consumption_utility(
        model_params, model_spec, states, covariates
    )

    return log_wage_systematic, non_consumption_utility


def calculate_log_wage_systematic(model_params, states, covariates, is_expected):
    """Calculate systematic wages, i.e., wages net of shock, for all states."""

    exp_p, exp_f = states[:, 3], states[:, 4]
    educ_level = covariates[:, 0]

    # Construct wage components
    gamma_0s = np.array(model_params.gamma_0s)[educ_level]
    gamma_1s = np.array(model_params.gamma_1s)[educ_level]

    if is_expected:
        period_exp_sum = 0.5 * exp_p + exp_f
    else:
        period_exp_sum = exp_p * np.array(model_params.g_s)[educ_level] + exp_f

    depreciation = 1 - np.array(model_params.delta_s)[educ_level]

    # Calculate wage in the given state
    period_exp_total = period_exp_sum * depreciation + 1
    returns_to_exp = gamma_1s * np.log(period_exp_total)
    log_wage_systematic = gamma_0s + returns_to_exp

    return log_wage_systematic


def calculate_non_consumption_utility(model_params, model_spec, states, covariates):
    """Calculate non-pecuniary utility contribution."""

    non_consumption_utility = np.full(
        (states.shape[0], NUM_CHOICES), [0, model_params.const_p, model_params.const_f]
    )
    
    # Type contribution
    for i in range(1, model_spec.num_types):
        non_consumption_utility[np.where(states[:, 5] == i)] += [
            0,
            model_params.theta_p[i - 1],
            model_params.theta_f[i - 1],
        ]
    
    ## TO DO: Introduce as free paramters in model spec
    # No children
    non_consumption_utility[np.where(covariates[:, 1] == 0)] += [
        0,
        0.320,
        -0.200,
    ]
    
    # Children present:
    non_consumption_utility[np.where(covariates[:, 1] != 0)] += [
        0,
        0.300,
        -0.175,
    ]
    
    # Contribution child aged 0-2:
    non_consumption_utility[np.where(covariates[:, 1] == 1)] += [
        0,
        0.156,
        -0.095,
    ]
    
    # Contribution child aged 3-5:
    non_consumption_utility[np.where(covariates[:, 1] == 2)] += [
        0,
        0.093,
        -0.067,
    ]
        
        # Contribution child aged 6-10:
    non_consumption_utility[np.where(covariates[:, 1] == 3)] += [
        0,
        0.047,
        -0.027,
    ]
    
    non_consumption_utility = np.exp(non_consumption_utility)

    return non_consumption_utility


In [17]:
log_wage_systematic, non_consumption_utilities = calculate_utility_components(
    model_params, model_spec, states, covariates, is_expected
)

In [18]:
non_consumption_utilities

array([[ 1.        ,  9.20733087,  9.40272932],
       [ 1.        , 10.54867226,  8.76704671],
       [ 1.        ,  8.33113749,  6.9657132 ],
       ...,
       [ 1.        ,  7.3890561 ,  5.84741685],
       [ 1.        ,  7.3890561 ,  5.84741685],
       [ 1.        ,  7.3890561 ,  5.84741685]])

Create draws
----------------

Draws for emax

In [19]:
from soepy.shared.shared_auxiliary import draw_disturbances

In [20]:
attrs_spec = ["seed_emax", "num_periods", "num_draws_emax"]
draws_emax = draw_disturbances(
    *[getattr(model_spec, attr) for attr in attrs_spec], model_params
)

Draws for exogenous process kids

In [34]:
# Idea: Determine age of the child in the child nodes outside of the get_continuation_values function
# since ages are a deterministic function of current age of child and the child_arriving variable

# Child arrives is equivalent to new child age = 0
# If no child arrives we need to specify an update rule
# Age stays at -1 if no kids so far
child_age_update_rule = np.full(states.shape[0], -1)
# Age increases by one, if there is a kid
child_age_update_rule[np.where(covariates[:, 1] != 0)] = states[np.where(covariates[:, 1] != 0)][:, 6] + 1
# Age does not exceed 11.
child_age_update_rule[child_age_update_rule > 11] = 11

Backward induction
------------------------

In [39]:
def pyth_backward_induction(
    model_spec, states, indexer, log_wage_systematic, non_consumption_utilities, draws, child_age_update_rule
):
    """Get expected maximum value function at every state space point.
    Backward induction is performed all at once for all states in a given period.
    The function loops through each period. The included construct_emax function
    implicitly loops through all states in the period currently reached by the
    parent loop.

    Parameters
    ----------
    model_spec : namedtuple
        Contains all fixed parameters of the model including information on dimensions
        such as number of periods, agents, random draws, etc.
    states : np.ndarray
        Array with shape (num_states, 5) containing period, years of schooling,
        the lagged choice, the years of experience in part-time, and the
        years of experience in full-time employment.
    indexer : np.ndarray
        Array where each dimension represents a componenet of the state space.
        :data:`states[k]` returns the values of the state space components
        at state :data:`k`. Indexing :data:`indexer` by the same state space
        component values returns :data:`k`.
    log_wage_systematic : np.array
        One dimensional array with length num_states containing the part of the wages
        at the respective state space point that do not depend on the agent's choice,
        nor on the random shock.
    non_consumption_utilities : np.ndarray
        Array of dimension (num_states, num_choices) containing the utility
        contribution of non-pecuniary factors.

    Returns
    -------
    emaxs : np.ndarray
        An array of dimension (num_states, num choices + 1). The object's rows contain
        the continuation values of each choice at the specific state space points
        as its first elements. The last row element corresponds to the maximum
        expected value function of the state.
    """

    emaxs = np.zeros((states.shape[0], NUM_CHOICES + 1))

    # Loop backwards over all periods
    for period in reversed(range(model_spec.num_periods)):

        # Extract period information
        states_period = states[np.where(states[:, 0] == period)]
        # Maybe faster like this
        #indexer_next_period = indexer[np.where(states[:, 0] == period + 1)]
        #emaxs_next_period = indexer[np.where(states[:, 0] == period + 1)]
        # Add info on updated age of child
        child_age_update_rule_period = child_age_update_rule[np.where(states[:, 0] == period)]
        
        log_wage_systematic_period = log_wage_systematic[states[:, 0] == period]
        non_consumption_utilities_period = non_consumption_utilities[
            states[:, 0] == period
        ]

        # Continuation value calculation not performed for last period
        # since continuation values are known to be zero
        if period == model_spec.num_periods - 1:
            pass
        else:

            # Fill first block of elements in emaxs for the current period
            # corresponding to the continuation values
            emaxs = get_continuation_values(model_spec, states_period, indexer, emaxs, child_age_update_rule_period)

        # Extract current period information for current loop calculation
        emaxs_period = emaxs[np.where(states[:, 0] == period)]

        # Calculate emax for current period reached by the loop
        emax_period = construct_emax(
            model_spec.delta,
            log_wage_systematic_period,
            non_consumption_utilities_period,
            draws[period],
            emaxs_period[:, :3],
            HOURS,
            model_spec.mu,
            model_spec.benefits,
        )
        emaxs_period[:, 3] = emax_period
        emaxs[np.where(states[:, 0] == period)] = emaxs_period

    return emaxs


@numba.njit(nogil=True)
def get_continuation_values(model_spec, states_subset, indexer, emaxs, child_age_update_rule):
    """Obtain continuation values for each of the choices at each state
    of the period currently reached by the parent loop.

    This function takes a parent node and looks up the continuation values
    of each of the available choices. It takes the entire block of
    data:`emaxs` corresponding to the period and fills in the first block
    of elements corresponding to the continuation values.
    The continuation value of each choice is the expected maximum value
    function of the next period's state if the particular choice was
    taken this period. The expected maximum value function values are
    contained as the last element of the data:`emaxs` row of next
    period's state.

    Warning
    -------
    This function must be extremely performant as the lookup is done for each state in a
    state space (except for states in the last period) for each evaluation of the
    optimization of parameters.
    """
    for i in range(states_subset.shape[0]):
        
        # Unpack parent state and get index
        period, educ_years, choice_lagged, exp_p, exp_f, type_, age_kid = states_subset[i]
        k_parent = indexer[
            period, educ_years - model_spec.educ_min, choice_lagged, exp_p, exp_f, type_, age_kid,
        ]
        
        # To Do: Add second state and integrate out probability
        # Choice: Non-employment
        k_0 = indexer[
            period + 1, 
            educ_years - model_spec.educ_min, 
            0, 
            exp_p, 
            exp_f, 
            type_, 
            child_age_update_rule[i],
        ]
        
        k_1 = indexer[
            period + 1, 
            educ_years - model_spec.educ_min, 
            0, 
            exp_p, 
            exp_f, 
            type_, 
            0,
        ]
        
        emaxs[k_parent, 0] = 0.5*emaxs[k_0, 3] + 0.5*emaxs[k_1, 3]

        # Choice: Part-time
        k_0 = indexer[
            period + 1, 
            educ_years - model_spec.educ_min, 
            1, 
            exp_p + 1, 
            exp_f, 
            type_, 
            child_age_update_rule[i],
        ]
        
        k_1 = indexer[
            period + 1, 
            educ_years - model_spec.educ_min, 
            1, 
            exp_p + 1, 
            exp_f, 
            type_, 
            0,
        ]
            
        emaxs[k_parent, 1] = 0.5*emaxs[k_0, 3] + 0.5*emaxs[k_1, 3]

        # Choice: Full-time
        k_0 = indexer[
            period + 1, 
            educ_years - model_spec.educ_min, 
            2, 
            exp_p, 
            exp_f + 1, 
            type_, 
            child_age_update_rule[i],
        ]
        
        k_1 = indexer[
            period + 1, 
            educ_years - model_spec.educ_min, 
            2, 
            exp_p, 
            exp_f + 1, 
            type_, 
            0,
        ]
        
        emaxs[k_parent, 2] = 0.5*emaxs[k_0, 3] + 0.5*emaxs[k_1, 3]

    return emaxs


@numba.njit
def _get_max_aggregated_utilities(
    delta,
    log_wage_systematic,
    non_consumption_utilities,
    draws,
    emaxs,
    hours,
    mu,
    benefits,
):

    current_max_value_function = INVALID_FLOAT

    for j in range(NUM_CHOICES):

        wage = np.exp(log_wage_systematic + draws[j])

        if j == 0:
            consumption_utility = benefits ** mu / mu
        else:
            consumption_utility = (hours[j] * wage) ** mu / mu

        value_function_choice = (
            consumption_utility * non_consumption_utilities[j] + delta * emaxs[j]
        )

        if value_function_choice > current_max_value_function:
            current_max_value_function = value_function_choice

    return current_max_value_function


@numba.guvectorize(
    ["f8, f8, f8[:], f8[:, :], f8[:], f8[:], f8, f8, f8[:]"],
    "(), (), (n_choices), (n_draws, n_choices), (n_choices), (n_choices), (), () -> ()",
    nopython=True,
    target="parallel",
)
def construct_emax(
    delta,
    log_wage_systematic,
    non_consumption_utilities,
    draws,
    emaxs,
    hours,
    mu,
    benefits,
    emax,
):
    """Simulate expected maximum utility for a given distribution of the unobservables.

    The function calculates the maximum expected value function over the distribution of
    the error term at each state space point in the period currently reached by the
    parent loop. The expectation calculation is performed via `Monte Carlo
    integration`. The goal is to approximate an integral by evaluating the integrand at
    randomly chosen points. In this setting, one wants to approximate the expected
    maximum utility of a given state.

    Parameters
    ----------
    delta : int
        Dynamic discount factor.
    log_wage_systematic : array
        One dimensional array with length num_states containing the part of the wages
        at the respective state space point that do not depend on the agent's choice,
        nor on the random shock.
    non_consumption_utilities : np.ndarray
        Array of dimension (num_states, num_choices) containing the utility
        contribution of non-pecuniary factors.
    draws : np.ndarray
        Array of dimension (num_periods, num_choices, num_draws). Randomly drawn
        realisations of the error term used to integrate out the distribution of
        the error term.
    emaxs : np.ndarray
        An array of dimension (num. states in period, num choices + 1).
        The object's rows contain the continuation values of each choice at the specific
        state space points as its first elements. The last row element corresponds
        to the maximum expected value function of the state. This column is
        full of zeros for the input object.
    hours : np.array
        Array of constants, corresponding to the working hours associated with
        each employment choice.
    mu : int
        Constant governing the degree of risk aversion and inter-temporal
        substitution in the model.
    benefits : int
        Constant level of hourly income received in case of choice N,
        non-employment.

    Returns
    -------
    emax : np.array
        Expected maximum value function of the current state space point.
        Array of lentgh number of states in the current period. The vector
        corresponds to the second block of values in the data:`emaxs` object.

    .. _Monte Carlo integration:
        https://en.wikipedia.org/wiki/Monte_Carlo_integration

    """
    num_draws = draws.shape[0]

    emax[0] = 0.0

    for i in range(num_draws):

        max_total_utility = _get_max_aggregated_utilities(
            delta,
            log_wage_systematic,
            non_consumption_utilities,
            draws[i],
            emaxs,
            hours,
            mu,
            benefits,
        )

        emax[0] += max_total_utility

    emax[0] /= num_draws

In [40]:
# Backward induction is now super super slow
emaxs = pyth_backward_induction(
    model_spec,
    states,
    indexer,
    log_wage_systematic,
    non_consumption_utilities,
    draws_emax,
    child_age_update_rule,
)

In [43]:
%timeit pyth_backward_induction(model_spec, states, indexer, log_wage_systematic, non_consumption_utilities, draws_emax, child_age_update_rule)

13.9 s ± 97.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
